In [67]:
import numpy as np

class State:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.board = np.zeros([3, 3]) # board entries 0 for empty, 1,2 for players 1,2
        self.result = 0 # 1 for p1, 2 for p2, -1 for draw, 0 for ongoing game
        self.turn = 1
        
    def legal_actions(self):
        if self.result != 0:
            return []
        
        board = self.board.reshape(-1)
        indices = np.array(range(len(board)), dtype=int)
        return indices[board == 0]
    
    def update(self, action):
        # assumed action is legal
        self.board[action//3, action%3] = self.turn
        self.turn = 3 - self.turn # turn changes even if game has ended but doesnt matter

        for p in range(1, 3):
            for i in range(3):
                if (self.board[i, :] == p).all():
                    self.result = p
            for j in range(3):
                if (self.board[:, j] == p).all():
                    self.result = p
            
            if self.board[0, 0] == self.board[1, 1] == self.board[2, 2] == p:
                self.result = p
            if self.board[0, 2] == self.board[1, 1] == self.board[2, 0] == p:
                self.result = p
            
        if self.result == 0 and (self.board.reshape(-1) != 0).all():
            self.result = -1
        
        return self.get_index()
    
    def get_index(self):
        board = self.board.reshape(-1)
        res = 0
        for i in range(len(board)):
            res = res*3 + int(board[i])
        return res
    
    def get_reward(self):
        if self.result == 0:
            return 0
        elif self.result == -1:
            return 0.5
        else:
            return 1
    
    def print_board(self):
        for i in range(3):
            for j in range(3):
                c = self.board[i, j]
                c = 'X' if c == 1 else 'O' if c == 2 else '-'
                print(c, end=' ')
            print('')
        print('====================================')
            
from collections import deque
class Agent:
    def __init__(self, alpha, gamma, epsilon, epsilon_min, epsilon_decay):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.reset()
        
    def reset(self):
        self.Q = np.full([3**9, 9], 0.0)
        self.memory = deque(maxlen = 10)
        
    def choose_action(self, state, debug=False):
        state_index = state.get_index()
        legal_actions = state.legal_actions()
        #print(legal_actions) #debug
        if np.random.random() <= self.epsilon:
            action = legal_actions[np.random.randint(len(legal_actions))]
        else:
            actions = np.full(9, -5.0)
            for i in legal_actions:
                actions[i] = self.Q[state_index, i]
            if debug:
                print(actions)
            action = np.argmax(actions)
#             action = np.random.choice(np.flatnonzero(actions == actions.max()))
        return action
    
    def remember(self, sarsA):
        self.memory.append(sarsA)
        
    def replay(self):
        for state_index, action, reward, next_state_index, legal_actions \
                in reversed(self.memory):
            self.update_Q(state_index, action, reward, next_state_index, legal_actions)
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
        self.memory.clear()
    
    def update_Q(self, state_index, action, reward, next_state_index, legal_actions):
        self.Q[state_index, action] = (1 - self.alpha) * self.Q[state_index, action] + \
                                         self.alpha * reward
        if len(legal_actions) != 0:
            self.Q[state_index, action] += self.alpha * self.gamma * \
                                            np.max(self.Q[next_state_index, legal_actions])
        
class Environment:
    def __init__(self, ALPHA=0.01, GAMMA=1.0, EPSILON=1.0, EPSILON_DECAY=0.995, LAST_EPSILON=0):
        EPSILON_MIN = 0.01
        self.epsilon = EPSILON
        self.last_epsilon = LAST_EPSILON
        self.alpha = ALPHA
        self.gamma = GAMMA
        self.p1 = Agent(ALPHA, GAMMA, EPSILON, EPSILON_MIN, EPSILON_DECAY)
        self.p2 = Agent(ALPHA, GAMMA, EPSILON, EPSILON_MIN, EPSILON_DECAY)
        self.state = State()
        
    def reset(self):
        self.p1.reset()
        self.p2.reset()
        self.state.reset()
        
    def learn(self, num_episodes):
        self.p1.epsilon = self.p2.epsilon = self.epsilon
        rs = np.zeros([10])
        for i in range(num_episodes):
            self.state.reset()
            prev_p2_state = None
            while True:
                prev_p1_state = self.state.get_index()
                p1_action = self.p1.choose_action(self.state)
                now_p2_state = self.state.update(p1_action)
                reward = self.state.get_reward()
                if prev_p2_state is not None:
                    legal_actions = self.state.legal_actions()
                    self.p2.remember((prev_p2_state, p2_action, reward if reward != 1 else -1,
                                      now_p2_state, legal_actions))
                if reward != 0:
                    self.p1.remember((prev_p1_state, p1_action, reward, None, []))
                    break
                    
                prev_p2_state = now_p2_state
                p2_action = self.p2.choose_action(self.state)
                now_p1_state = self.state.update(p2_action)
                reward = self.state.get_reward()
                legal_actions = self.state.legal_actions()
                self.p1.remember((prev_p1_state, p1_action, reward if reward != 1 else -1,
                                  now_p1_state, legal_actions))
                if reward != 0:
                    self.p2.remember((prev_p2_state, p2_action, reward, None, []))
                    break
            
            self.p1.replay()
            self.p2.replay()
            if num_episodes - i < 10:
                self.p1.epsilon = self.p2.epsilon = self.last_epsilon
                rs[num_episodes - i] = self.state.result
            
        first_won = sum((rs == 1).astype(int))
        second_won = sum((rs == 2).astype(int))
        draw = sum((rs == -1).astype(int))
        return (first_won, second_won, draw)                    

In [68]:
def train_env(env, i=500, j=800):
    from tqdm.notebook import trange
    for i in trange(i):
        print(env.learn(j))

In [79]:
def playFirst(p2):
        # player first
    game = State()
    game.print_board()
    while True:
        action = int(input())
        #state_index = self.state.get_index()
        next_state_index = game.update(action)
        game.print_board()
        reward = game.get_reward()
        #legal_actions = self.state.legal_actions()
        if reward != 0:
            break
        action = p2.choose_action(game, debug=False)
        #state_index = game.get_index()
        next_state_index = game.update(action)
        game.print_board()
        reward = game.get_reward()
        if reward != 0:
            break
    print("Game over.")
    print("You won" if game.result == 1 else "CPU won" if game.result == 2 else "Draw")
    
def playSecond(p1):
    game = State()
    game.print_board()
    while True:
        action = p1.choose_action(game, debug=False)
        #state_index = game.get_index()
        next_state_index = game.update(action)
        game.print_board()
        reward = game.get_reward()
        if reward != 0:
            break
        print()
        action = int(input())
        #state_index = self.state.get_index()
        next_state_index = game.update(action)
        game.print_board()
        reward = game.get_reward()
        #legal_actions = self.state.legal_actions()
        if reward != 0:
            break
    print("Game over.")
    print("You won" if game.result == 2 else "CPU won" if game.result == 1 else "Draw")
   

In [80]:
import copy
def getAgents(env):
    p1 = copy.deepcopy(env.p1)
    p1.epsilon = 0
    cnt = 0
#     for i in range(3**9):
#         arr = p1.Q[i] != 0
#         if arr.any():
#             cnt += 1
#     print(cnt)
    p2 = copy.deepcopy(env.p2)
    p2.epsilon = 0
    return p1, p2

In [81]:
env = Environment(ALPHA=0.01, EPSILON=0.3, GAMMA=0.7, EPSILON_DECAY=1.0)

Widget Javascript not detected.  It may not be installed or enabled properly.


(1, 0, 8)
(0, 0, 9)
(0, 0, 9)
(0, 0, 9)
(0, 0, 9)
(0, 0, 9)
(1, 0, 8)
(1, 0, 8)
(1, 0, 8)
(1, 0, 8)

Type 1 for first player. 2 for second player
0
- - - 
- - - 
- - - 
X - - 
- - - 
- - - 

0
O - - 
- - - 
- - - 
O X - 
- - - 
- - - 

2
O X O 
- - - 
- - - 
O X O 
X - - 
- - - 

4
O X O 
X O - 
- - - 
O X O 
X O X 
- - - 

6
O X O 
X O X 
O - - 
Game over.
You won
Another game? y/n
n


In [ ]:
train_env(env, 50, 800) # change these numbers... it shows progress bar with 500 iterations,
                         # each iteration is 800 games... basically 500*800 games training

In [ ]:
p1, p2 = getAgents(env)
while True:
    print("Type 1 for first player. 2 for second player")
    ch = int(input())
    if ch == 1:
        playFirst(p2)
    else:
        playSecond(p1)
    print("Another game? y/n")
    ch = input()
    if ch == 'n' or ch == 'N':
        break